In [ ]:
# install dependencies: 
!pip -q install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from detectron2.structures import BoxMode
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
train_df=pd.read_csv('../input/med-mech-obj-det-dataset/annotations.csv',)

In [ ]:
train_df["file"]="../input/med-mech-obj-det-dataset/2D_images/2D_images/"+train_df["file"]

In [ ]:
val_df=train_df[3300:]
train_df=train_df[:3300]

In [ ]:
dataframes={
    "train":train_df,
    "eval":val_df
}

In [ ]:
def get_data_dict(df):
    list_of_images=list(df["file"].unique())
    dataset_dicts=list()
    for idx,image_path in enumerate(tqdm(list_of_images)):
        record=dict()
        image_df=df.loc[df["file"]==image_path]
        record["file_name"]=image_path
        record["image_id"]=idx
        record["height"]=image_df.iloc[0,1].astype(int)
        record["width"]=image_df.iloc[0,2].astype(int)
        objs=list()
        for _, row in image_df.iterrows():
            bbox=list((int(row[3]),int(row[5]),int(row[4]),int(row[5])))
            obj={
                "bbox":bbox,
                 "bbox_mode":BoxMode.XYXY_ABS,
                 "category_id":0
            }
            objs.append(obj)
        record["annotations"]=objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
for mode in ["train","eval"]:
    DatasetCatalog.register(f"seed_{mode}", lambda mode=mode:get_data_dict(dataframes[mode]))
    MetadataCatalog.get(f"seed_{mode}").set(thing_classes=["SEED"])

In [ ]:
seed_metadata=MetadataCatalog.get("seed_val")

In [ ]:
dataset_dicts=get_data_dict(val_df)
for item in random.sample(dataset_dicts,3):
    img=cv2.imread(item["file_name"])
    print (item["annotations"])
    visualizer = Visualizer(img[:, :,  ::-1], metadata=seed_metadata, scale=0.5)
    out=visualizer.draw_dataset_dict(item)
    plt.imshow(out.get_image()[: :, ::-1])
    plt.show()

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
cfg.MODEL.MASK_ON=False
cfg.MODEL.WEIGHTS = "../input/mcmed-cascade-rcnn/output/model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2

In [ ]:
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
# pred_dict={file_name:[],boxes:[],scores:[]}
for d in random.sample(dataset_dicts, 5):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=seed_metadata, 
                   scale=0.5)
    print(outputs["instances"].pred_boxes)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()